In [ ]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)

In [2]:
from quanta.clients.yfinance import YahooFinanceClient
from datetime import datetime, timedelta

n=365
from_date = datetime.now() - timedelta(days=n)
to_date = datetime.now() - timedelta(days=1)

yh = YahooFinanceClient()
df_init = yh.get_price(
    "^SPMCBRT",
    from_date=from_date.strftime("%Y-%m-%d"),
    to_date=to_date.strftime("%Y-%m-%d"),
    interval="1h", 
    postclean=True
)

df_init.to_pandas()

,timestamp,datetime,open,high,low,close,volume
0,1730903400,2024-11-06 15:30:00,1261.956909,1284.460327,1261.956909,1280.506226,0
1,1730907000,2024-11-06 16:30:00,1279.447144,1293.914307,1275.322266,1287.995850,0
2,1730910600,2024-11-06 17:30:00,1286.996704,1288.534058,1277.593994,1284.306641,0
3,1730914200,2024-11-06 18:30:00,1284.648193,1285.451050,1280.036499,1281.701782,0
4,1730917800,2024-11-06 19:30:00,1281.505371,1282.940063,1274.126709,1276.389771,0
...,...,...,...,...,...,...,...
1486,1762270200,2025-11-04 16:30:00,1245.520264,1250.555054,1245.044922,1248.071655,0
1487,1762273800,2025-11-04 17:30:00,1247.441040,1250.157349,1246.306152,1248.915649,0
1488,1762277400,2025-11-04 18:30:00,1248.226807,1251.515381,1246.257446,1249.963257,0
1489,1762281000,2025-11-04 19:30:00,1249.992432,1250.855835,1245.073975,1246.703857,0


In [3]:
# import strategy :
from quanta.ta_clients.optimization_strategy import StrategyClient
strat = StrategyClient()

/home/cmileto/miniconda3/envs/ta/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
INDICATOR_CLASSES = strat.get_seed_indicator_config()
INDICATOR_CLASSES


{'SMA': quanta.utils.ta.SMA,
 'RSI': quanta.utils.ta.RSI,
 'MACD': quanta.utils.ta.MACD,
 'BollingerBands': quanta.utils.ta.BollingerBands,
 'ATR': quanta.utils.ta.ATR}

In [5]:
OPTIMIZATION_CONFIG = strat.get_seed_optimization_config()
OPTIMIZATION_CONFIG

{'strategy_name': 'simple_strategy',
 'indicators': {'SMA_short': {'class': 'SMA',
   'params': {'period': {'type': 'int', 'low': 5, 'high': 50}}},
  'SMA_long': {'class': 'SMA',
   'params': {'period': {'type': 'int', 'low': 20, 'high': 200}}},
  'RSI': {'class': 'RSI',
   'params': {'period': {'type': 'int', 'low': 14, 'high': 21}}},
  'ATR': {'class': 'ATR',
   'params': {'period': {'type': 'int', 'low': 10, 'high': 30}}},
  'BollingerBands': {'class': 'BollingerBands',
   'params': {'period': {'type': 'int', 'low': 15, 'high': 30},
    'std_dev': {'type': 'float', 'low': 2.0, 'high': 3.0}}},
  'MACD': {'class': 'MACD',
   'params': {'fast_period': {'type': 'int', 'low': 8, 'high': 16},
    'slow_period': {'type': 'int', 'low': 20, 'high': 30},
    'signal_period': {'type': 'int', 'low': 7, 'high': 12}}}},
 'strategy': {'stop_loss': {'type': 'float',
   'low': 0.005,
   'high': 0.02,
   'log': True},
  'take_profit': {'type': 'float', 'low': 0.01, 'high': 0.05, 'log': True},
  'posi

In [6]:
strat.print_strategies()

Available strategies:

  simple_strategy:
    Description: Simple strategy based on SMA crossovers and RSI levels.
    Required indicators: ['SMA_short', 'SMA_long', 'RSI', 'MACD', 'ATR']


In [7]:
STRATEGY = strat.get_strategy_fct("simple_strategy")

In [8]:
# test module OptimizationClient : 
from quanta.ta_clients.optimization import OptimizationClient
opt = OptimizationClient(optimization_config=OPTIMIZATION_CONFIG)


ℹ️  Ignored indicators (not used by 'simple_strategy'): {'BollingerBands'}
ℹ️  Ignored constraint (indicator not used): RSI_period < BollingerBands_period
ℹ️  Ignored constraint (indicator not used): SMA_short_period < BollingerBands_period
✓ Strategy 'simple_strategy' initialized with indicators: ['SMA_short', 'SMA_long', 'RSI', 'MACD', 'ATR']


In [9]:
opt.optimize(
    df = df_init, 
    backtest_func=STRATEGY, 
    verbose=False)

[I 2025-11-06 17:39:03,168] A new study created in memory with name: trading_strategy_optimization
Best trial: 986. Best value: 31.5737: 100%|██████████| 1000/1000 [00:22<00:00, 44.77it/s, 22.33/3600 seconds]


In [10]:
opt.analyze_parameter_importance()
useless_params = opt.get_useless_parameters(threshold=0.01)
opt.plot_importance()  # Graphique interactif

⚠️  Pas assez de trials valides pour calculer l'importance des paramètres
❌ Impossible de calculer l'importance: Cannot evaluate parameter importances without completed trials.


In [11]:
from financeta.utils.ta import TAClient

indicators = opt.get_best_indicators()
indicators


In [12]:

ta_client = TAClient()
df = ta_client.calculate_indicators(df_init, indicators)

from financeta.utils.trace import Candlesticks, Volume, Line

traces = [
    Candlesticks(),
    Line('hl_avg', name='HL Average', color='purple'),
    Volume()
]


from financeta.clients.chart import ChartClient

chart_client = ChartClient()
chart_client.plot(df, "AAPL", traces=traces, indicators=indicators, theme='professional')

Plotting 1491 bars for AAPL with x_axis_type='row_nb'


In [13]:
trades = opt.get_trades_history(df=df_init)
trades

  Signaux générés: 622
  📊 Signaux totaux: 622 (Long: 274, Short: 348)
  💼 Trades exécutés: 622
  📅 Période: 2024-11-22 16:30:00 → 2025-10-30 19:30:00 (342 jours = 0.94 ans)
  📊 Trades: 622 | Return: 0.5174 | Sharpe: 33.87 | DD: 4.78%


timestamp,position_number,action,price,quantity_usd,position_size,pnl,cumulative_capital
datetime[μs],i64,str,f64,f64,f64,f64,f64
2024-11-22 17:30:00,0,"""SELL""",1274.837769,0.798795,0.799441,-0.000808,0.999192
2024-11-22 18:30:00,1,"""SELL""",1280.246582,0.796085,0.799441,-0.003392,0.995803
2024-11-22 19:30:00,2,"""SELL""",1277.653809,0.797374,0.799441,0.001619,0.997415
2024-11-22 20:30:00,3,"""SELL""",1277.851318,0.797276,0.799441,-0.000124,0.997292
2024-11-25 15:30:00,4,"""SELL""",1249.071533,0.811631,0.799441,0.018005,1.015248
…,…,…,…,…,…,…,…
2025-10-27 16:30:00,617,"""SELL""",1258.389038,1.315031,0.799441,0.00072,1.644938
2025-10-27 17:30:00,618,"""SELL""",1256.130615,1.316918,0.799441,0.001435,1.647298
2025-10-27 18:30:00,619,"""SELL""",1252.592651,1.319883,0.799441,0.002252,1.651007


In [14]:
chart_client = ChartClient()
chart_client.plot(
    df, "AAPL", 
    traces=traces, 
    indicators=indicators, 
    trades_df=trades, 
    theme='professional'
)

Plotting 1491 bars for AAPL with x_axis_type='row_nb'
With 622 trades
Debug: 622 trades après traitement
shape: (5, 4)
┌─────────────────────┬─────────┬────────┬─────────────┐
│ timestamp           ┆ x_value ┆ action ┆ price       │
│ ---                 ┆ ---     ┆ ---    ┆ ---         │
│ datetime[μs]        ┆ i64     ┆ str    ┆ f64         │
╞═════════════════════╪═════════╪════════╪═════════════╡
│ 2024-11-22 17:30:00 ┆ 92      ┆ SELL   ┆ 1274.837769 │
│ 2024-11-22 18:30:00 ┆ 93      ┆ SELL   ┆ 1280.246582 │
│ 2024-11-22 19:30:00 ┆ 94      ┆ SELL   ┆ 1277.653809 │
│ 2024-11-22 20:30:00 ┆ 95      ┆ SELL   ┆ 1277.851318 │
│ 2024-11-25 15:30:00 ┆ 96      ┆ SELL   ┆ 1249.071533 │
└─────────────────────┴─────────┴────────┴─────────────┘
